## Restarting

In [ ]:
%cd /content/drive/My Drive/20192/bigdata
import requests
from lxml import html
import os
import re
# crawled_url = set()
import csv
save_file_base = "news/soha_all_{}.csv"
root_site = "https://soha.vn/"
crawled_site_file = "crawled_site"
with open(crawled_site_file, 'r') as read:
    data = read.read().split('\n')
    crawled_url = set(data)
crawled_url
restart = True
if not restart:
    with open('init_site', 'r') as reader:
        start_urls = reader.read().split("\n")
else:
    with open('crawled_site', 'r') as reader:
        start_urls = reader.read().split("\n")[-10:]
i = 1 
while start_urls:
    curl = start_urls.pop()
    if curl not in crawled_url:
        i += 1
    if i % 50 == 0:
        print(f"crawled {i} site")
    # print(curl)
    news_id = curl[-21:-4]
    tail = news_id[:8]
    if not tail.isdigit():
        tail = "home"
    save_file = save_file_base.format(tail)
    if not os.path.isfile(save_file):
        with open(save_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["news_id", "toppic", "title", "description", "content", "links"])
    
    with open(crawled_site_file, 'a') as w:
        w.write(curl + "\n")
    response = requests.get(curl) 
    source = html.fromstring(response.content)

    try:
        topic = source.xpath('//*[@id="sohaSubCategories"]/a[1]/text()')[0]
    except:
        topic = curl
    try:

        title = source.xpath('//*[@id="admWrapsite"]/div[3]/div/div[6]/div[2]/div[1]/main/article/header/h1/text()')[0].\
                    replace("\r\n", '').strip()
    except:
        title = curl
    try:
        description = source.xpath('//*[@id="admWrapsite"]/div[3]/div/div[6]/div[2]/div[1]/\
                    main/article/div[1]/div[1]/h2/text()')[-1].replace("\r\n", '').strip()
    except:
        description = curl

    content_all = source.xpath('//*[@id="admWrapsite"]/div[3]/div/div[6]/div[2]/div[1]'
                                 '/main/article/div[1]/div[1]/div[4]/p')
    content = ""
    for c in content_all:
        docs = c.xpath('./text()')
        if not docs:
            docs = c.xpath('./span/text()')
        if docs:
            content += docs[0]
    if curl not in crawled_url:
        crawled_url.add(curl)
        with open(save_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([news_id, topic, title, description, content, curl])

    links = re.findall("data-popup-url=\"\/([\w\d-]*.htm)", str(response.content))
    links = set(links)
    for link in links:
        full_link = link
        if not link.startswith('https:/'):
            full_link = root_site + link
        if full_link not in crawled_url and full_link not in start_urls and \
         ('20200830' <= full_link[-21:-4] <= '20200928'):
            start_urls.insert(0, full_link)

/content/drive/My Drive/20192/bigdata
crawled 50 site
crawled 100 site
crawled 150 site
crawled 200 site
crawled 250 site
crawled 300 site
crawled 350 site
crawled 400 site
crawled 450 site
crawled 500 site
crawled 550 site
crawled 600 site
crawled 650 site
crawled 700 site
crawled 750 site
crawled 800 site
crawled 850 site
crawled 900 site
crawled 950 site
crawled 1000 site
crawled 1050 site
crawled 1100 site


In [ ]:
!wget https://seafile.platform.vn/f/e6a71af49cd14e64b2f9/?dl=1

--2020-10-13 08:14:54--  https://seafile.platform.vn/f/e6a71af49cd14e64b2f9/?dl=1
Resolving seafile.platform.vn (seafile.platform.vn)... 103.69.195.91
Connecting to seafile.platform.vn (seafile.platform.vn)|103.69.195.91|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://seafile.platform.vn/seafhttp/files/6dd1aabf-d5c8-4dfd-b517-91475727c61f/news_dataset.zip [following]
--2020-10-13 08:14:56--  https://seafile.platform.vn/seafhttp/files/6dd1aabf-d5c8-4dfd-b517-91475727c61f/news_dataset.zip
Reusing existing connection to seafile.platform.vn:443.
HTTP request sent, awaiting response... 200 OK
Length: 776991798 (741M) [application/zip]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 741.00M  11.9MB/s    in 76s     

2020-10-13 08:16:12 (9.73 MB/s) - ‘index.html?dl=1’ saved [776991798/776991798]



In [ ]:
!unzip soha

In [ ]:
!pip install Google-Search-API

In [ ]:
import pandas as pd

In [ ]:
data_0830 = pd.read_csv("/content/news/2020-08-30.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_0830.head()

,d_guid,guid,itemId,pageloadId,timestamp,tor
0,487df34ed78a314668f01419a51b5d93,4384389871241395155,/,1598793108068,1.598793e+09,59.0
1,487df34ed78a314668f01419a51b5d93,4384389871241395155,20200830162435216,1598793289933,1.598793e+09,19.0
2,487df34ed78a314668f01419a51b5d93,4384389871241395155,/,1598793609875,1.598794e+09,60.0
3,487df34ed78a314668f01419a51b5d93,4384389871241395155,20200830100133481,1598793858977,1.598794e+09,19.0
4,487df34ed78a314668f01419a51b5d93,4384389871241395155,/,1598793609875,1.598794e+09,60.0


In [ ]:
!pip install git+https://github.com/abenassi/Google-Search-API

In [ ]:
from googlesearch import search 

def get_link(news_id):
    url = ""
    if not news_id.isdigit():
        print("")
        pass
    else:
        try:
            query = "soha" + str(news_id)
            url = search(query, num=1).__next__()
        except:
            url = ""
    print(f"{news_id} ==>link: {url}")
    return url

In [ ]:
from google import google
num_page = 3
search_results = google.search("soha ")

In [ ]:
import os
import pandas as pd

for file in os.listdir("/content/news"):
    data = pd.read_csv(f"/content/news/{file}")
    all_item = list(set(data["itemId"]))
    links = pd.DataFrame({"item_id": all_item})
    links['link'] = links['item_id'].map(lambda f: get_link(f))

    links.to_csv(f"/content/links/{file}_links.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


20191118140415993 ==>link: https://soha.vn/dien-vien-kim-dao-toi-dang-om-con-ma-ba-cam-cai-chao-tang-bop-vao-mat-trong-nha-toan-tieng-chui-boi-la-het-20191118140415993.htm
20160831095247629 ==>link: https://soha.vn/loi-ich-lon-nhat-khi-samsung-phat-trien-kinh-doanh-tai-viet-nam-20160831095247629.htm
2011092308016881 ==>link: https://soha.vn/xa-hoi/nhung-moc-tang-giam-ngoan-muc-cua-gia-vang-tu-dau-nam-2011-den-nay-2011092308016881.htm
2019080909432137 ==>link: https://soha.vn/khai-quat-mo-co-nghin-nam-cua-chau-gai-hoang-hau-trung-hoa-va-cau-chuyen-bi-an-dang-sau-4-chu-nguoi-mo-se-chet-tren-nap-quan-tai-2019080909432137.htm
20200805204025966 ==>link: https://soha.vn/muc-luong-tren-troi-cua-cac-vuong-gia-nha-thanh-co-tien-ti-moi-nam-van-pha-san-vi-1-thu-20200805204025966.htm
202009192153571 ==>link: 
20141107072209443 ==>link: https://soha.vn/cu-dan-mang/nu-mc-hai-phong-xinh-dep-duyen-dang-giua-tiet-troi-dau-dong-20141107072209443.htm
20141030165847137 ==>link: https://soha.vn/kinh-doanh/

In [ ]:
all_item

In [ ]:
get_link("20200830162435216")

'https://soha.vn/con-gai-dai-gia-bat-dong-san-trung-quoc-co-sac-voc-xuat-chung-nhung-lan-dan-tinh-duyen-26-tuoi-da-co-trong-tay-ca-doi-bong-20200830162435216.htm'

In [ ]:
from googlesearch import search 

In [ ]:
a = search("soha 20200830162435216", num=1).__next__()

## Crawl link sapo search 

In [ ]:
!wget https://seafile.platform.vn/f/e6a71af49cd14e64b2f9/?dl=1
!unzip -q "index.html?dl=1"

--2020-10-14 01:22:33--  https://seafile.platform.vn/f/e6a71af49cd14e64b2f9/?dl=1
Resolving seafile.platform.vn (seafile.platform.vn)... 103.69.195.91
Connecting to seafile.platform.vn (seafile.platform.vn)|103.69.195.91|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://seafile.platform.vn/seafhttp/files/22aaf2b1-5203-42ec-b2e8-8590e3a2167c/news_dataset.zip [following]
--2020-10-14 01:22:34--  https://seafile.platform.vn/seafhttp/files/22aaf2b1-5203-42ec-b2e8-8590e3a2167c/news_dataset.zip
Reusing existing connection to seafile.platform.vn:443.
HTTP request sent, awaiting response... 200 OK
Length: 776991798 (741M) [application/zip]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 741.00M  16.7MB/s    in 64s     

2020-10-14 01:23:38 (11.6 MB/s) - ‘index.html?dl=1’ saved [776991798/776991798]



In [ ]:
import pandas as pd
from googlesearch import search 
from multiprocessing import Pool
from time import time
import csv
import os


save_link_file = "/content/drive/My Drive/links"
if not os.path.isfile(save_link_file):
    with open(save_link_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["news_id", "link"])

save_item_got = "/content/drive/My Drive/item_got"
if os.path.isfile(save_item_got):
    with open(save_item_got, "r") as reader:
        item_got = reader.readline().split("\n")[:-1]
else:
    item_got = []
item_got = set(item_got)


def get_link(news_id):
    url = ""
    if news_id not in item_got:
        if news_id.isdigit():
            query = "soha " + str(news_id)
            url = search(query, num=1).__next__()
        with open(save_link_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([news_id, url])
        with open(save_item_got, "a") as file_dict:
            file_dict.write(news_id + "\n")
    # print(f"{news_id} ==>link: {url}")
    return url

all_links = []
for file in os.listdir("/content/news"):
    if file.endswith(".csv"):
        data = pd.read_csv(f"/content/news/{file}")
        all_item = set(data["itemId"])
        all_item = all_item - item_got
        all_item = list(all_item)

        n = len(all_item)
        step = 50
        k = int(n/step)
        for i in range(k+1):
            with Pool(step) as p:
                all_link = p.map(get_link, all_item[i*step:(i+1)*step])
            all_links.extend(all_link)
            del p

In [ ]:
import multiprocessing
import pandas as pd
from googlesearch import search 
from multiprocessing import Pool
from time import time
import csv
import os


save_link_file = "/content/drive/My Drive/links"
if not os.path.isfile(save_link_file):
    with open(save_link_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["news_id", "link"])

save_item_got = "/content/drive/My Drive/item_got"
if os.path.isfile(save_item_got):
    with open(save_item_got, "r") as reader:
        item_got = reader.readline().split("\n")[:-1]
else:
    item_got = []
item_got = set(item_got)

def worker(news_id):
    url = ""
    if news_id not in item_got:
        if news_id.isdigit():
            query = "soha " + str(news_id)
            url = search(query, num=1).__next__()
        with open(save_link_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([news_id, url])
        with open(save_item_got, "a") as file_dict:
            file_dict.write(news_id + "\n")
    print(f"{news_id} ==>link: {url}")
    # return url
    exit(0)


all_links = []
for file in os.listdir("/content/news"):
    if file.endswith(".csv"):
        data = pd.read_csv(f"/content/news/{file}")
        all_item = set(data["itemId"])
        all_item = all_item - item_got
        all_item = list(all_item)

        n = len(all_item)
        step = 50
        k = int(n/step)
        for i in range(k+1):
            # with Pool(step) as p:
            #     all_link = p.map(get_link, all_item[i*step:(i+1)*step])
            # all_links.extend(all_link)
            jobs = []
            for j in range(i*step, min((i+1)*step, n)):
                p = multiprocessing.Process(target=worker, args=(all_item[j],))
                jobs.append(p)
                p.start()
            result = []
            for proc in jobs:
                proc.join()
                result.append(proc.exitcode)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


20200804111301763 ==>link: https://soha.vn/trung-quoc-set-danh-trung-toa-nha-gay-mua-tia-lua-20200804111301763.htm
20190418114856414 ==>link: https://soha.vn/dong-y-khen-cay-dai-nay-tot-tu-goc-den-ngon-la-qua-re-deu-co-the-nau-thanh-mon-an-thuoc-20190418114856414.htm
2020090609162564 ==>link: https://soha.vn/hoi-uc-cong-vinh-bi-fan-viet-thoa-ma-va-cach-xu-ly-di-vao-long-nguoi-cua-clb-nhat-ban-2020090609162564.htm
20180627143504606 ==>link: https://soha.vn/chuyen-gia-chi-ra-8-cach-day-con-da-loi-thoi-cha-me-nen-thay-doi-ngay-20180627143504606.htm
20190425065858723 ==>link: https://soha.vn/suc-manh-khung-khiep-cua-ho-den-quai-vat-vu-tru-vua-sa-luoi-nhan-loai-20190425065858723.htm
20200716115131736 ==>link: https://soha.vn/bam-huyet-moi-ngay-huyet-phong-tri-cai-thien-ngay-nhung-can-benh-o-vung-dau-co-20200716115131736.htm
20140125151517534 ==>link: https://soha.vn/the-gioi-do-day/cac-loai-banh-co-truyen-trong-ngay-tet-cua-cac-nuoc-20140125151517534.htm
20200914101048615 ==>link: https://s

Process Process-997:


20170529094800765 ==>link: https://soha.vn/dien-vien-bao-thanh-thien-sau-24-nam-nguoi-dong-phim-nong-vi-ngheo-kho-ke-giau-co-nhung-co-don-20170529094800765.htm
20140725112340005 ==>link: https://soha.vn/news-20140725112340005.htm


Traceback (most recent call last):


...

Streaming output truncated to the last 5000 lines.
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "<ipython-input-3-7be2c72ffc05>", line 29, in worker
    url = search(query, num=1).__next__()
Traceback (most recent call last):
  File "<ipython-input-3-7be2c72ffc05>", line 29, in worker
    url = search(query, num=1).__next__()
  File "<ipython-input-3-7be2c72ffc05>", line 29, in worker
    url = search(query, num=1).__next__()
  File "/usr/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    se

/bat-dong-san ==>link: 


Process Process-1687:
Process Process-1699:
Process Process-1665:
Process Process-1674:
Process Process-1666:
Process Process-1654:
Process Process-1697:
Process Process-1694:
Process Process-1681:
Process Process-1672:
Process Process-1671:
Process Process-1685:
Process Process-1691:
Process Process-1675:
Process Process-1660:
Process Process-1657:
Process Process-1653:
Process Process-1667:
Process Process-1693:
Process Process-1684:
Process Process-1700:
Process Process-1677:
Process Process-1673:
Process Process-1658:
Process Process-1652:
Process Process-1655:
Process Process-1695:
Process Process-1670:
Process Process-1664:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-1692:
Process Process-1698:
Process Process-1689:
Process Process-1683:
Process Process-1663:
Process Process-1656:
Process Process-1682

KeyboardInterrupt: ignored

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258,

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/crawl_soha/crawl_news"

/content/drive/My Drive/Colab Notebooks/crawl_soha/crawl_news


In [ ]:
with open("/content/drive/My Drive/item_got", "r") as reader:
    data0 = reader.read().split("\n")
x = len(data0)
y = len(set(data0))
x, y

(13645, 9093)

In [ ]:
len(set(pd.read_csv(save_link_file)['news_id'])), len(pd.read_csv(save_link_file)['news_id'])

(9095, 13657)

In [ ]:
# if __name__ == '__main__':
#     jobs = []
#     for i in range(5):
#         p = multiprocessing.Process(target=worker, args=(i,))
#         jobs.append(p)
#         p.start()

#     result = []
#     for proc in jobs:
#         proc.join()
#         result.append(proc.exitcode)
#     print result

## 24h crawler

In [ ]:
import requests
import re

In [ ]:
text = requests.get("https://www.24h.com.vn/#").text

In [ ]:
links = re.findall("href=\"(https://www.24h.com.vn/[a-z0-9.:/-]+)\"", text)

In [ ]:
links

In [ ]:
import requests
import re
starting = False
init_link = []
save_link_path = "/content/drive/My Drive/20192/news/crawl_24h/links"
with open(save_link_path, 'r') as reader:
    init_link = reader.read().split("\n")

crawled_link = set(init_link)
queue_link = set()
if starting:
    queue_link = {"https://www.24h.com.vn/#"}
else:
    queue_link = set(init_link[-500:])

def crawl_link_24h(url):
    try:
        text = requests.get(url).text
    except:
        text = ""
    links = re.findall("href=\"(https://www.24h.com.vn/[a-z0-9.:/-]+)\"", text)
    return set(links)

count = len(init_link)
while len(queue_link) > 0:
    url = queue_link.pop()
    crawl_link = crawl_link_24h(url)
    news_link = list(crawl_link-crawled_link)
    count += len(news_link)
    print(f"Crawled {count} links!")
    with open(save_link_path, 'a') as writer:
        for link in news_link:
            writer.write(link+"\n")
    crawled_link = crawled_link.union(news_link)
    queue_link = queue_link.union(news_link)


Streaming output truncated to the last 5000 lines.
Crawled 87855 links!
Crawled 87859 links!
Crawled 87861 links!
Crawled 87864 links!
Crawled 87866 links!
Crawled 87870 links!
Crawled 87871 links!
Crawled 87874 links!
Crawled 87877 links!
Crawled 87880 links!
Crawled 87883 links!
Crawled 87885 links!
Crawled 87888 links!
Crawled 87890 links!
Crawled 87890 links!
Crawled 87893 links!
Crawled 87897 links!
Crawled 87899 links!
Crawled 87902 links!
Crawled 87905 links!
Crawled 87908 links!
Crawled 87912 links!
Crawled 87915 links!
Crawled 87918 links!
Crawled 87922 links!
Crawled 87924 links!
Crawled 87929 links!
Crawled 87932 links!
Crawled 87937 links!
Crawled 87943 links!
Crawled 87946 links!
Crawled 87949 links!
Crawled 87949 links!
Crawled 87952 links!
Crawled 87956 links!
Crawled 87958 links!
Crawled 87960 links!
Crawled 87960 links!
Crawled 87962 links!
Crawled 87965 links!
Crawled 87966 links!
Crawled 87967 links!
Crawled 87967 links!
Crawled 87970 links!
Crawled 87971 links!
Craw

In [ ]:
a.pop()

1

In [ ]:
a

{2, 3}

In [ ]:
!wget https://download.informer.com/win-1191519231-efd8210-680ad9b3/virtualbox-5.1.28-117968-win.exe

--2020-10-24 15:02:47--  https://download.informer.com/win-1191519231-efd8210-680ad9b3/virtualbox-5.1.28-117968-win.exe
Resolving download.informer.com (download.informer.com)... 199.101.132.53
Connecting to download.informer.com (download.informer.com)|199.101.132.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123929944 (118M) [application/x-dosexec]
Saving to: ‘virtualbox-5.1.28-117968-win.exe’

              virtu   2%[                    ]   3.07M  76.5KB/s    eta 25m 18s